In [53]:
import json
import os
import shutil
import glob
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model
from keras.preprocessing.image import img_to_array
import cv2

In [ ]:
# specify th paths
os.mkdir('objects')
os.mkdir('merged')
os.mkdir('day')
os.mkdir('obj_labels')

In [74]:
label_path = 'archive/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json'
image_path = 'archive/bdd100k/bdd100k/images/100k/'

new_path = 'merged/'
day_path = 'day/'
objects_path = 'objects/'
obj_labels = 'obj_labels/'

In [59]:
# Move all images in image_path to new_path
for img in glob.glob(image_path + '/**/*.jpg', recursive = True):
    shutil.move(img, new_path)

In [7]:
# load labels
f = open(label_path)
train = json.load(f)

## Select Images Including Our Objects

In [43]:
# select images including traffic_lights

categories = ['traffic sign', 'traffic light']
objects = {}

for item in train:
    name = item['name']
    
    # this loops checks the existence of all our custom objects in an image,
    # if there are, it creates an empty list for that image and breaks
    obj_list = []
    for label in item['labels']:
        if label['category'] in categories:
            obj_list.append(label['category'])
            if set(obj_list) == set(categories):
                objects[name] = []
                break

In [44]:
# It takes a while!
for key in objects.keys():
    for item in train:
        if key == item['name']:
            for label in item['labels']:
                
                if label['category'] == categories[0]:
                    label['box2d']['id'] = 0
                    bbox = label['box2d']
                    objects[key].append(bbox)
                    
                if label['category'] == categories[1]:
                    label['box2d']['id'] = 1
                    bbox = label['box2d']
                    objects[key].append(bbox)

In [62]:
# move them to another folder
for img in objects:
    shutil.move(os.path.join(new_path+img), os.path.join(objects_path,img))

## Day-Night Classification

In [66]:
model = load_model('day_night.hd5')

In [67]:
objs = objects

In [68]:
# It takes a while!
# 0 = night , 1 = day
for img in os.listdir(objects_path):
    
    image = cv2.imread(objects_path+img)
    image = cv2.resize(image, (28, 28))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    label = model.predict(image)
    
    if label[0][0] > label[0][1]:
        objs[img].append(0)
    else:
        objs[img].append(1)

2021-12-04 11:08:22.127260: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-04 11:08:23.211424: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


In [69]:
obj = objs

# Remove night images from dic
for key, value in obj.copy().items():
    if value[-1] == 0:
        del obj[key]
        
# Move day images to another folder        
for key in obj:
    shutil.move(objects_path+key, day_path)

## Convert Annotation

In [70]:
# remove label
for value in obj.values():
    del value[-1]
    
# save annotations
with open('objects.json', 'w') as fp:
    json.dump(obj, fp)

In [71]:
f = open('objects.json')
train = json.load(f)

In [77]:
# It takes a while!
# [xmin, ymin, xmax, ymax] > [xcenter, ycenter, objwidth, objheight]
for name, value in train.items():
    # check image sizes
    im = cv2.imread(day_path+name)
    h, w, c = im.shape
    
    for i, obj in enumerate(value):
        value[i] = [ obj['id'], (obj['x2'] + obj['x1']) / (2*w), (obj['y2'] + obj['y1']) / (2*h), (obj['x2'] - obj['x1']) / w, (obj['y2'] - obj['y1']) / h]

In [79]:
# save them
fmt = '%.10g'

for name, obj in train.items():
    np.savetxt(obj_labels+name.replace(name[-4:],'.txt'), obj, newline='\n',fmt=fmt)